# Modelos de regresión en SKlearn
## 1. Objetivo

Familiarizarse con los conceptos de modelos de regresion

## Datos de Melb Houses
* Suburb: Suburb
* Address: Address
* Rooms: Number of rooms
* Price: Price in Australian dollars
* Method:
  * S - property sold;
  * SP - property sold prior;
  * PI - property passed in;
  * PN - sold prior not disclosed;
  * SN - sold not disclosed;
  * NB - no bid;
  * VB - vendor bid;
  * W - withdrawn prior to auction;
  * SA - sold after auction;
  * SS - sold after auction price not disclosed.
  * N/A - price or highest bid not available.

* Type:
  * br - bedroom(s);
  * h - house,cottage,villa, semi,terrace;
  * u - unit, duplex;
  * t - townhouse;
  * dev site - development site;
  * res - other residential.

* SellerG: Real Estate Agent

* Date: Date sold

* Distance: Distance from CBD in Kilometres
* Regionname: General Region (West, North West, North, North east …etc)
* Propertycount: Number of properties that exist in the suburb.
* Bedroom2 : Scraped # of Bedrooms (from different source)
* Bathroom: Number of Bathrooms
* Car: Number of carspots
* Landsize: Land Size in Metres
* BuildingArea: Building Size in Metres
* YearBuilt: Year the house was built
* CouncilArea: Governing council for the area
* Lattitude: Self explanitory
* Longtitude: Self explanitory


* Melb Data: https://www.kaggle.com/code/dansbecker/handling-missing-values/data?select=melb_data.csv
* Sklearn Cheat Sheet from Datacamp: https://media.datacamp.com/legacy/image/upload/v1676302389/Marketing/Blog/Scikit-Learn_Cheat_Sheet.pdf

## 2. Librerias de trabajo

In [1]:
# Instala libreria Pandas si no la tenemos
#pip install pandas seaborn scikit-learn -y

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import (
    LinearRegression,
    Lasso,
    Ridge,
    ElasticNet
)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import (
    LabelEncoder,
    OrdinalEncoder,
    LabelBinarizer,
    OneHotEncoder
)

from sklearn.linear_model import (
    LinearRegression,
    Lasso,
    Ridge,
    ElasticNet
)

from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

## 3. Lectura de datos

Primero nos encargaremos de leer los datos, indicando a Python donde se encuentra la carpeta que contiene los datos y los nombres de los archivos relevantes para el análisis.

In [3]:
#  Indicamos la ruta a la carpeta de de tu computadora 
# donde se ubican los datos del E-commerce
# Ejemplo: "C:\Usuarios\[tu nombre]\Descargas"

DATA_PATH="../data"

Ahora procederemos a definir una variable que indique el nombre del archivo junto con su extensión (por ejemplo, `.csv`):

In [4]:
FILE_DATA_PATH = "melb_data.csv"

Echaremos mano de la utilidad `os.path.join` de Python que indicar rutas en tu computadora donde se ubican archivos, así Pandas encontrá los archivos de datos.


**Ejemplo**

A continuación mostraremos un ejemplo leyendo el archivo `melb_data.csv`.
csv`:

In [5]:
# Ejemplo
print(f"Ruta del archivo: {FILE_DATA_PATH}")
print(os.path.join(DATA_PATH, FILE_DATA_PATH))

Ruta del archivo: melb_data.csv
../data/melb_data.csv


In [6]:
# Leemos con pandas
df = pd.read_csv(
    os.path.join(DATA_PATH, FILE_DATA_PATH)
    )

In [7]:
df.sample(10)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
8475,Viewbank,23 Kuranga Rd,4,h,1140000,S,Miles,08/04/17,13.8,3084,...,2,4.0,863,NaN,NaN,Banyule,-37.73170,145.09520,Eastern Metropolitan,2698
5527,South Melbourne,18 Cobden St,2,h,920500,S,Greg,03/12/16,2.1,3205,...,1,0.0,114,NaN,NaN,Port Phillip,-37.83440,144.96860,Southern Metropolitan,5943
12141,Richmond,712/8 Howard St,2,u,927000,S,Biggin,29/07/17,2.4,3121,...,1,2.0,0,NaN,2000.0,Yarra,-37.83305,144.99745,Northern Metropolitan,14949
3462,Keilor East,117 Brees Rd,3,h,976000,S,Nelson,12/11/16,12.8,3033,...,3,2.0,605,212.0,1980.0,Moonee Valley,-37.74590,144.87880,Western Metropolitan,5629
8629,Keilor Park,7 Zagreb Ct,3,h,773000,S,Nelson,29/04/17,14.5,3042,...,2,2.0,650,150.0,1980.0,Brimbank,-37.72400,144.85140,Western Metropolitan,1119
3784,Malvern,51 Dixon St,2,h,1611000,S,Marshall,12/11/16,7.4,3144,...,1,2.0,277,NaN,NaN,Stonnington,-37.85860,145.03750,Southern Metropolitan,4675
4254,Newport,94 Mason St,2,h,801000,SP,Williams,18/06/16,8.4,3015,...,1,1.0,343,106.0,1920.0,Hobsons Bay,-37.84140,144.87520,Western Metropolitan,5498
4101,Moonee Ponds,22 McPherson St,4,h,1310000,S,Jellis,16/04/16,6.9,3039,...,3,2.0,475,NaN,NaN,Moonee Valley,-37.76090,144.92850,Western Metropolitan,6232
11171,Preston,66 McNamara St,4,h,1148000,S,Nelson,12/08/17,8.4,3072,...,1,2.0,552,140.0,1940.0,Darebin,-37.72879,144.98538,Northern Metropolitan,14577
6307,Thornbury,2/104 Keon St,2,u,450000,PI,Nelson,22/08/16,6.5,3071,...,1,1.0,335,59.0,1960.0,Darebin,-37.75160,144.98710,Northern Metropolitan,8870


Revisemo las información de los datos:

In [8]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  str    
 1   Address        13580 non-null  str    
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  str    
 4   Price          13580 non-null  int64  
 5   Method         13580 non-null  str    
 6   SellerG        13580 non-null  str    
 7   Date           13580 non-null  str    
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  int64  
 10  Bedroom2       13580 non-null  int64  
 11  Bathroom       13580 non-null  int64  
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  int64  
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  str    
 17  Lattitude      13580 non-null  float64
 18  Longtitude     13

## 2. Modelos Básicos de Aprendizaje de Máquina

Esta sección aborda algunos de los principales modelos usados en problemas de aprendizaje supervisado.

### 2.1 Modelos de Regresión Lineal

Los modelos de regresión son modelos estadísticos que asumen la existencia de una relación lineal entre las características del problema y la variable a predecir, es decir, la variable objetivo se puede describir de la forma

$$y_i = \beta_0 + \beta_1 x_{i1}  + \beta_2 x_{i2} + \ldots + \beta_n x_{in}+ \epsilon_i$$

Donde los coeficientes $\beta_i$ son parámetros por determinarse y $\epsilon_i$ es una representación del ruido en la relación entre $y_i$ y $x_{i1}, \ldots, x_{in}$. En la jerga estadística los variables reciben el nombre de regresores.

Debe notarse que dado que los parámetros deben estimarse existen supuestos de tipo estadístico para que la relación lineal entre las variables y el objetivo se aproxime de buena manera, es decir, el residuno, nombre que recib el error entre $y$ y la $x_{i1}, \ldots, x_{1n}$, que generalmente se describe en termino del error cuadrático medio.

La teoría estadística señala algunos de los supuestos que deben cumplirse para que la regresión sea un modelo con buen funcionamiento (es decir controlando el sesgo y la varianza de la predicción):

* **Linealidad:** La relación entre la variable dependiente $Y$ y las variables independientes $X$ debe ser lineal.
* **Normalidad de los residuos:** Los residuos deben seguir una distribución normal, este supuesto se puede relajar a una distribución aproximadamente normal.
* **Homogeneidad de la varianza de los residuos:** Los residuos deben tener una varianza constante (homocedasticidad), es decir se espera que la variación de los residuos no se dispare en alguna región del espacio.
* **Independencia de los residuos:** Los residuos deben ser independientes los unos de los otros, es decir  no deben estar correlacionados entre sí.

Por otro lado la estimación de los coeficientes $\beta_i$ se realiza con técnicas de análisis numérico y que siguen principios estadísticos que, cumpliendose los supuestos anteriores, aseguran que el modelo obtenido tenga la combinación de coeficientes que asegura una varianza mínimo (Teorema de Gauss-Markov). Normalmente los paquetes de cómputo científico realizan los cálculos correspondientes, pero la verificación de dichos supuestos es responsabilidad de quien realiza el análisis.

Es dable mencionar que los modelos de regresión son herramientas poderosas de aprendizaje supervisado, pero tienen algunos puntos a considerar:

* Se ven afectados fuertemente por la presencia de valores atípicos y la presencia de escala distintas en los datos, por lo que es común pre-procesar los datos para eliminar valores ruidosos y asegurando que todos sus componentes tengan órdenes de magnitud comparables.
* Requieren que los variables dentro de los mismos sean linealmente independientes en el sentido del álgebra lineal, normalmente un buen análisis de correlación puede ayudar a desechar aquellas variables entre las que exista correlación lineal para mejorar su desempeño
* Son muy flexibles, típicamente puede transformarse para capturar relaciones no lineales.
* Generalmente la cantidad de variables que se puede incluir está limitada por la cantidad de puntos en el conjunto de entrenamiento.
* Si todas las variables tienen la misma escala, los coeficientes se pueden interpretar como el efecto que tiene una característica sobre la variable objetivo, considerando que todas las demás características se quedan fija (*ceteris paribus*).

#### 2.1.1 Modelos de Regresión Lineal y Regularización

Para mejorar evitar el sobreajuste de estos modelos, existen técnicas que penalizan la complejidad del modelo, es decir el valor que pueden tomar los coeficientes y que en general inducen mejores resultados para predecir.

Los modelos de regresión que incluyen penalizaciones sobre los coeficientes tienen denominaciones especiales:

* **Lasso:** Este modelo es similar al descrito arriba pero considerando que los coefientes se encuentre en una región definida por $\sum |\beta_i| \leq K_1$ donde $K_1$ es alguna constante. Esta métrica generalmente hace que algunos de los coeficientes $\beta_i$ sean cercanos a cero o bien se anulen.
* **Ridge:** Similar al modelo previo, pero considerando que los coefientes se encuentre en una región definida por $\sum |\beta_i|^2 \leq K_2$ donde $K_2$ es alguna constante.
* **Elastic Net:** En este caso, se pide que los coeficientes satisfagan la restricción $\sum |\beta_i| + \sum |\beta_i|^2 \leq K_3$ donde $K_3$ es cierta constante.


Una forma alternativa de describir lo anterior, es la formulación de Lagrange los problemas de minimización con restriciones, donde esencialmente se busca minimizar la expresión para encontrar el error más el valor de los parámetros

* **Lasso:** $$||y - \beta X||^2_2 + \alpha ||\beta||^2_2$$ donde el coeficiente $\alpha$ es un parámetros que controla la restricción del tamaño de $\beta$. 
* **Ridge:** $$||y - \beta X||^2_2 + \alpha ||\beta||^2_1$$ donde el coeficiente $\alpha$ es un parámetros que controla la restricción del tamaño de $\beta$. 
* **Elastic NEt:** $$||y - \beta X||^2_2 + \alpha_1 ||\beta||^2_1 + \alpha_2  ||\beta||^2_2$$ donde el coeficiente $\alpha$ es un parámetros que controla la restricción del tamaño de $\beta$. (La parametrización de Sklear es equivalente, ver https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet)

En Sklearn, dichos modelos se encuentran disponibles en las clases `LinearRegression`, `Lasso`, `Ridge` y `ElasticNet`. En Python, los valores de $\alpha_i$ y sus reparamtrizaciónes se utilizan para controlar que tantos se restringe a los coeficientes para controlar el sobreajuste. Al ser parámetros que no dependen específicamente de los datos, de denominar **hiper-parámetros**.

# 3. Modelos de regresión para predecir el precio de Melb Houses

In [9]:
# Define listas de columnas que van a emplearse en el modelado
num_features = [
    'Rooms', 
    'BuildingArea',
    'Landsize',
    'Distance',
    'Bathroom',
    'YearBuilt'
 ]

cat_cols = ['Regionname', 'Type']

# Lista que tiene todas los grupos de columnas
non_target_cols = num_features + cat_cols

target = ['Price']

In [10]:
df["BuildingArea"] = df["BuildingArea"].fillna(0)

In [11]:
df["YearBuilt"] = df["YearBuilt"].fillna(df["YearBuilt"].median())

In [12]:
df[non_target_cols].isna().sum()

Rooms           0
BuildingArea    0
Landsize        0
Distance        0
Bathroom        0
YearBuilt       0
Regionname      0
Type            0
dtype: int64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    df[non_target_cols],
    df[target],
    test_size=0.2,
)

In [14]:
# Pipeline para escalar con estandar z-score
numerical_pipe = Pipeline([
    ('standar_scaler', StandardScaler())
])

categorical_pipe = Pipeline([
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
])

In [15]:
# Combina ambos procesos en columnas espeficadas en listas
pre_processor = ColumnTransformer([
    ('numerical', numerical_pipe, num_features),
    ('categorical', categorical_pipe, cat_cols),
], remainder='passthrough')

## 4. Models

### 4.1 Linear Regresion

In [45]:
lr = LinearRegression()

In [24]:
X_train.info()

<class 'pandas.DataFrame'>
Index: 10864 entries, 2162 to 6579
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rooms         10864 non-null  int64  
 1   BuildingArea  10864 non-null  float64
 2   Landsize      10864 non-null  int64  
 3   Distance      10864 non-null  float64
 4   Bathroom      10864 non-null  int64  
 5   YearBuilt     10864 non-null  float64
 6   Regionname    10864 non-null  str    
 7   Type          10864 non-null  str    
dtypes: float64(3), int64(3), str(2)
memory usage: 763.9 KB


In [46]:
lr.fit(X_train[["Rooms","Landsize","Bathroom"]], y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [47]:
lr.get_params()

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'positive': False,
 'tol': 1e-06}

In [48]:
lr.coef_[0]

array([2.21158112e+05, 3.02205196e+00, 2.51993424e+05])

In [49]:
lr.intercept_

array([33987.99698666])

In [57]:
print("LinearRegression Coefficients: ")
pd.concat(
[
    pd.DataFrame(
    {
        "coef_names": "coef_" + lr.feature_names_in_,
        "coef_values": lr.coef_[0]
    }
    ),
    pd.DataFrame(
    {
        "coef_names": ["intercept"],
        "coef_values": lr.intercept_
    }
    )
]
)

LinearRegression Coefficients: 


,coef_names,coef_values
0,coef_Rooms,221158.112255
1,coef_Landsize,3.022052
2,coef_Bathroom,251993.423883
0,intercept,33987.996987


### 4.2 Ridge Regresion

In [51]:
ridge = Ridge(alpha=0.23)

In [52]:
ridge.fit(X_train[["Rooms","Landsize","Bathroom"]], y_train)

,"alpha alpha: {float, ndarray of shape (n_targets,)}, default=1.0Constant that multiplies the L2 term, controlling regularizationstrength. `alpha` must be a non-negative float i.e. in `[0, inf)`.When `alpha = 0`, the objective is equivalent to ordinary leastsquares, solved by the :class:`LinearRegression` object. For numericalreasons, using `alpha = 0` with the `Ridge` object is not advised.Instead, you should use the :class:`LinearRegression` object.If an array is passed, penalties are assumed to be specific to thetargets. Hence they must correspond in number.",0.23
,"fit_intercept fit_intercept: bool, default=TrueWhether to fit the intercept for this model. If setto false, no intercept will be used in calculations(i.e. ``X`` and ``y`` are expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"max_iter max_iter: int, default=NoneMaximum number of iterations for conjugate gradient solver.For 'sparse_cg' and 'lsqr' solvers, the default value is determinedby scipy.sparse.linalg. For 'sag' solver, the default value is 1000.For 'lbfgs' solver, the default value is 15000.",None
,"tol tol: float, default=1e-4The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for each solver:- 'svd': `tol` has no impact.- 'cholesky': `tol` has no impact.- 'sparse_cg': norm of residuals smaller than `tol`.- 'lsqr': `tol` is set as atol and btol of scipy.sparse.linalg.lsqr, which control the norm of the residual vector in terms of the norms of matrix and coefficients.- 'sag' and 'saga': relative change of coef smaller than `tol`.- 'lbfgs': maximum of the absolute (projected) gradient=max|residuals| smaller than `tol`... versionchanged:: 1.2 Default value changed from 1e-3 to 1e-4 for consistency with other linear models.",0.0001
,"solver solver: {'auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs'}, default='auto'Solver to use in the computational routines:- 'auto' chooses the solver automatically based on the type of data.- 'svd' uses a Singular Value Decomposition of X to compute the Ridge coefficients. It is the most stable solver, in particular more stable for singular matrices than 'cholesky' at the cost of being slower.- 'cholesky' uses the standard :func:`scipy.linalg.solve` function to obtain a closed-form solution.- 'sparse_cg' uses the conjugate gradient solver as found in :func:`scipy.sparse.linalg.cg`. As an iterative algorithm, this solver is more appropriate than 'cholesky' for large-scale data (possibility to set `tol` and `max_iter`).- 'lsqr' uses the dedicated regularized least-squares routine :func:`scipy.sparse.linalg.lsqr`. It is the fastest and uses an iterative procedure.- 'sag' uses a Stochastic Average Gradient descent, and 'saga' uses its improved, unbiased version named SAGA. Both methods also use an iterative procedure, and are often faster than other solvers when both n_samples and n_features are large. Note that 'sag' and 'saga' fast convergence is only guaranteed on features with approximately the same scale. You can preprocess the data with a scaler from :mod:`sklearn.preprocessing`.- 'lbfgs' uses L-BFGS-B algorithm implemented in :func:`scipy.optimize.minimize`. It can be used only when `positive` is True.All solvers except 'svd' support both dense and sparse data. However, only'lsqr', 'sag', 'sparse_cg', and 'lbfgs' support sparse input when`fit_intercept` is True... versionadded:: 0.17 Stochastic Average Gradient descent solver... versionadded:: 0.19 SAGA solver.",'auto'
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive.Only 'lbfgs' solver is supported in this case.",False
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag' or 'saga' to shuffle the data.See :term:`Glossary ` for details... versionadded:: 0.17 `random_state` to support Stochastic Average Gradient.",None


In [55]:
ridge.coef_

array([2.21157549e+05, 3.02211772e+00, 2.51982661e+05])

In [56]:
ridge.intercept_[0]

np.float64(34006.09159342456)

In [58]:
print("Ridge Regression Coefficients: ")
pd.concat(
[
    pd.DataFrame(
    {
        "coef_names": "coef_" + lr.feature_names_in_,
        "coef_values": lr.coef_[0]
    }
    ),
    pd.DataFrame(
    {
        "coef_names": ["intercept"],
        "coef_values": lr.intercept_
    }
    )
]
)

Ridge Regression Coefficients: 


,coef_names,coef_values
0,coef_Rooms,221158.112255
1,coef_Landsize,3.022052
2,coef_Bathroom,251993.423883
0,intercept,33987.996987


### 4.3 Lasso Regresion

In [59]:
lasso = Lasso(alpha=0.023)

In [60]:
lasso.fit(X_train[["Rooms","Landsize","Bathroom"]], y_train)

,"alpha alpha: float, default=1.0Constant that multiplies the L1 term, controlling regularizationstrength. `alpha` must be a non-negative float i.e. in `[0, inf)`.When `alpha = 0`, the objective is equivalent to ordinary leastsquares, solved by the :class:`LinearRegression` object. For numericalreasons, using `alpha = 0` with the `Lasso` object is not advised.Instead, you should use the :class:`LinearRegression` object.",0.023
,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"precompute precompute: bool or array-like of shape (n_features, n_features), default=FalseWhether to use a precomputed Gram matrix to speed upcalculations. The Gram matrix can also be passed as argument.For sparse input this option is always ``False`` to preserve sparsity.",False
,"copy_X copy_X: bool, default=TrueIf ``True``, X will be copied; else, it may be overwritten.",True
,"max_iter max_iter: int, default=1000The maximum number of iterations.",1000
,"tol tol: float, default=1e-4The tolerance for the optimization: if the updates are smaller or equal to``tol``, the optimization code checks the dual gap for optimality and continuesuntil it is smaller or equal to ``tol``, see Notes below.",0.0001
,"warm_start warm_start: bool, default=FalseWhen set to ``True``, reuse the solution of the previous call to fit asinitialization, otherwise, just erase the previous solution.See :term:`the Glossary `.",False
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive.",False
,"random_state random_state: int, RandomState instance, default=NoneThe seed of the pseudo random number generator that selects a randomfeature to update. Used when ``selection`` == 'random'.Pass an int for reproducible output across multiple function calls.See :term:`Glossary `.",None
,"selection selection: {'cyclic', 'random'}, default='cyclic'If set to 'random', a random coefficient is updated every iterationrather than looping over features sequentially by default. This(setting to 'random') often leads to significantly faster convergenceespecially when tol is higher than 1e-4.",'cyclic'


In [61]:
lasso.coef_

array([2.21158105e+05, 3.02205224e+00, 2.51993381e+05])

In [62]:
lasso.intercept_[0]

np.float64(33988.082636631676)

In [63]:
print("Lasso Regression Coefficients: ")
pd.concat(
[
    pd.DataFrame(
    {
        "coef_names": "coef_" + lr.feature_names_in_,
        "coef_values": lr.coef_[0]
    }
    ),
    pd.DataFrame(
    {
        "coef_names": ["intercept"],
        "coef_values": lr.intercept_
    }
    )
]
)

Lasso Regression Coefficients: 


,coef_names,coef_values
0,coef_Rooms,221158.112255
1,coef_Landsize,3.022052
2,coef_Bathroom,251993.423883
0,intercept,33987.996987


In [17]:
models_regresion = {
    'regression': LinearRegression(),
    'lasso_01': Lasso(alpha=0.1),
    'lasso_001': Lasso(alpha=0.01),
    'lasso_0001': Lasso(alpha=0.001),
    'lasso_000001': Lasso(alpha=0.0001),
    'ridge_01': Ridge(alpha=0.1),
    'ridge_001': Ridge(alpha=0.01),
    'ridge_0001': Ridge(alpha=0.001),
    'ridge_000001': Ridge(alpha=0.0001),
    'elastic_05_03': ElasticNet(alpha=0.5, l1_ratio=0.3),
    'elastic_0001_01': ElasticNet(alpha=0.0001, l1_ratio=0.0001)
    }

In [18]:
models = []
models_train_errors = []
models_test_errors = []

for model_name in models_regresion.keys():

    print("Modelo:", model_name)
    model = Pipeline([
        ('transform', pre_processor),
        ('model', models_regresion[model_name])
    ])

    # Ajusta el modelo con los datos de prueba
    model.fit(X_train[non_target_cols],y_train)

    y_train_pred = model.predict(X_train[non_target_cols])
    y_test_pred = model.predict(X_test[non_target_cols])

    # error en conjunto de entrenamiento y prueba
    error_train = root_mean_squared_error(y_train, y_train_pred)
    error_test = root_mean_squared_error(y_test, y_test_pred)

    # errores
    print("Error RSME en train:", round(error_train,4) )
    print("Error RSME en test:", round(error_test,4) )

    print("----------------------------------------------")

    models.append(model_name)
    models_train_errors.append(error_train)
    models_test_errors.append(error_test)

    

Modelo: regression
Error RSME en train: 405028.6221
Error RSME en test: 429942.7981
----------------------------------------------
Modelo: lasso_01
Error RSME en train: 405028.6221
Error RSME en test: 429942.8839
----------------------------------------------
Modelo: lasso_001
Error RSME en train: 405028.6221
Error RSME en test: 429942.8067
----------------------------------------------
Modelo: lasso_0001
Error RSME en train: 405028.6221
Error RSME en test: 429942.7989
----------------------------------------------
Modelo: lasso_000001
Error RSME en train: 405028.6221
Error RSME en test: 429942.7982
----------------------------------------------
Modelo: ridge_01
Error RSME en train: 405028.6254
Error RSME en test: 429944.4534
----------------------------------------------
Modelo: ridge_001
Error RSME en train: 405028.6221
Error RSME en test: 429942.9635
----------------------------------------------
Modelo: ridge_0001
Error RSME en train: 405028.6221
Error RSME en test: 429942.8146
---

In [19]:
pd.DataFrame({
    "model": models,
    "rmse_train": models_train_errors,
    "rmse_test": models_test_errors,
}).sort_values(["rmse_test"])

,model,rmse_train,rmse_test
0,regression,405028.622083,429942.798089
4,lasso_000001,405028.622083,429942.798175
3,lasso_0001,405028.622083,429942.798948
8,ridge_000001,405028.622083,429942.799743
2,lasso_001,405028.622083,429942.806670
7,ridge_0001,405028.622083,429942.814626
1,lasso_01,405028.622097,429942.883910
6,ridge_001,405028.622116,429942.963472
5,ridge_01,405028.625393,429944.453422
10,elastic_0001_01,405029.012769,429960.818534


In [20]:
models_regresion["regression"].coef_

array([[ 169764.50109787,   13878.4043542 ,   14612.60739657,
        -232620.67895312,  157485.81291654,  -69648.43799237,
         -30885.64661124,  276692.24735665, -259533.82377201,
          61014.16081421,  139836.99470582,  211902.18001293,
        -312207.04023614,  -86819.07227023,  253499.30651428,
         -29675.56050191, -223823.74601237]])

In [21]:
models_regresion["lasso_001"].coef_

array([ 169764.47300133,   13878.42180233,   14612.60623134,
       -232620.51661798,  157485.84128261,  -69648.4488117 ,
         43818.02324219,  351392.84734611, -184829.89819427,
        135714.11591745,  214540.16709297,  286605.98409162,
       -237503.18256679,  -12111.78813337,  429721.68145299,
        146546.69336465,  -47601.28391976])